In [3]:
%load_ext autoreload

%autoreload 2

In [4]:

import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import math
import matplotlib
import matplotlib.pyplot as plt
import random

from src.tf_tools.rnn_regression import coolRNN
from src.tf_tools.cnn_regression import originalResNet
from src.grammar.sentences import sentences_Cspacer

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# 3.10.4

In [22]:
keras.backend.clear_session()
model = originalResNet()
model.summary()

Model: "originalResNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 164, 4)]             0         []                            
                                                                                                  
 zero_padding1d (ZeroPaddin  (None, 182, 4)               0         ['input_1[0][0]']             
 g1D)                                                                                             
                                                                                                  
 conv1d (Conv1D)             (None, 182, 128)             5248      ['zero_padding1d[0][0]']      
                                                                                                  
 A_bn (BatchNormalization)   (None, 182, 128)             512       ['conv1d[0][0]'] 

In [ ]:
def one_hot_seqs(seqs) -> np.array:
    static_1hotmap = {
        'A' : np.array([1,0,0,0]),
        'a' : np.array([1,0,0,0]),
        'C' : np.array([0,1,0,0]),
        'c' : np.array([0,1,0,0]),
        'G' : np.array([0,0,1,0]),
        'g' : np.array([0,0,1,0]),
        'T' : np.array([0,0,0,1]),
        't' : np.array([0,0,0,1]),
    }
    onehot_seqs = []
    for seq in seqs:
        onehot_seqs.append(
            [static_1hotmap[seq[i]] if seq[i] in static_1hotmap.keys() else static_1hotmap[random.choice(['A','C','G','T'])] for i in range(len(seq))]
        )
    return np.stack(onehot_seqs)

In [ ]:
activity_df = pd.read_csv('Data/activity.csv', index_col=0)
activity_df = activity_df.set_index('label')
activity_df['sentence'] = sentences_Cspacer(activity_df.index.to_list(), "Data/Motifs/summarized_motifs.parquet")

In [ ]:
FEATURE_KEY = 'sequence'
LABEL_KEY = 'expression_log2'